## LSTM and Music Generation

Credits: https://towardsdatascience.com/how-to-generate-music-using-a-lstm-neural-network-in-keras-68786834d4c5

### RNN: Classic Unrolling
![rnn](https://chunml.github.io/images/projects/creating-text-generator-using-recurrent-neural-network/vanilla_RNN.png)



Hidden layer at time step $t$ ($h_t$), is computed from using $h_{t-1}$ (from previous time step) ($\sigma$ = sigmoid):

$$h_t=\sigma(W_{xh}x_t+W_{hh}h_{t−1})$$

Output (only based on latest $h_t$):

$$y_t = softmax(W_{hy}h_t)$$

### RNN problem: Vanishing Gradient
- As we back-propagate, gradient of the cost function w.r.t. weights tends to diminish.
- This means we forget stuff from earlier time steps.
- Training becomes very slow because RNN doesn't remember much 

![vanishing gradient](https://cdn-images-1.medium.com/max/2000/1*FWy4STsp8k0M5Yd8LifG_Q.png)

### LSTM

![lstm](https://chunml.github.io/images/projects/creating-text-generator-using-recurrent-neural-network/LSTM.png)

- $h_{t-1}$ is the output at time step $t-1$
- Cell state ($C_t$) holds the "long-short term memory" and is controlled by 3 gates:
  - Input gate: decides which values to update ($i_t$)
  - Forget gate: decides which values to forget ($f_t$)
  - Output gate: decides which values to output ($o_t$)
  
- $h_t$ is the output at time step $t$

## Attention vs. LSTM / RNNs

In recent months, LSTMs and RNNs have fallen out of favour (machine learning is like the fashion industry). 

We won't address this in this workbook, but may cover it in the future.

Some background if you are curious:

https://towardsdatascience.com/the-fall-of-rnn-lstm-2d1594c74ce0

https://arxiv.org/abs/1502.03044

https://github.com/philipperemy/keras-attention-mechanism

## Music Generation

Train a neural network to generate midi files

Repository: https://github.com/Skuldur/Classical-Piano-Composer

Blog post: https://towardsdatascience.com/how-to-generate-music-using-a-lstm-neural-network-in-keras-68786834d4c5

### Setup

[Music21](http://web.mit.edu/music21/doc/about/what.html) is a Python toolkit for computer-aided musicology (study of music, editing and composing music)

Install to your environment:

```
(mldds03) pip install music21
```


In [1]:
# clone the repository
!git clone https://github.com/Skuldur/Classical-Piano-Composer

'git' is not recognized as an internal or external command,
operable program or batch file.


### Predict

As a starting point, we won't modify the script or setup, but try out the demo to make sure it still works.

Later on, we can add our own midi files, tweak the LSTM network, etc.

In [2]:
# !cd is needed because the ! syntax always evaluates from the current directory
!cd Classical-Piano-Composer & dir

 Volume in drive D is DATA
 Volume Serial Number is B200-6E0E

 Directory of D:\mldds-courseware\03_TextImage\Classical-Piano-Composer

04/08/2018  08:49 AM    <DIR>          .
04/08/2018  08:49 AM    <DIR>          ..
02/08/2018  02:43 PM                66 .gitattributes
02/08/2018  04:44 PM    <DIR>          data
04/08/2018  08:42 AM    <DIR>          logs
02/08/2018  02:43 PM             3,978 lstm.py
04/08/2018  08:42 AM             4,241 lstm_exercise.py
04/08/2018  08:41 AM    <DIR>          midi_songs
02/08/2018  05:25 PM    <DIR>          midi_songs2
04/08/2018  08:40 AM    <DIR>          midi_songs_bak
02/08/2018  02:43 PM             4,771 predict.py
02/08/2018  02:43 PM               917 README.md
02/08/2018  05:05 PM             8,536 test_output.mid
02/08/2018  02:55 PM             4,752 test_output.mid.back
04/08/2018  08:42 AM        44,081,840 weights-improvement-01-5.1664-bigger.hdf5
04/08/2018  08:43 AM        44,081,840 weights-improvement-02-5.0590-bigger.hdf5
04/08

There is a pre-trained LSTM network (`weights.hdf5`) already present. Let's run it to generate music.

In [ ]:
!cd Classical-Piano-Composer & python predict.py

In [4]:
# if you have Visual Studio Code installed, you can run this to inspect predict.py
# if you don't have Visual Studio Code installed, you can replace
# code with notepad or your text editor
!cd Classical-Piano-Composer & code predict.py


This produces the following file:

`test_output.mid`

In [5]:
# Reference: https://blog.ouseful.info/2016/09/13/making-music-and-embedding-sounds-in-jupyter-notebooks/

def play_midi(filename):
    """Plays a midi file
    Args:
        filename - path to the midi file
    """
    from music21 import midi
    
    mf = midi.MidiFile()
    mf.open(filename)
    mf.read()
    mf.close()
    stream = midi.translate.midiFileToStream(mf)
    stream.show('midi')

play_midi('Classical-Piano-Composer/test_output.mid')

### Train

Now that we've verified that the pre-trained network works, let's try training using our own midi files.

Note that the original network takes about 20 hours to train, so we'll just be training a smaller version of 
the network for demonstration purposes.

### Training with GPU

If you have a machine with an NVidia GPU, you can use keras-gpu to speed up training (about 6x speedup).

This requires uninstalling keras (cpu version) and install keras-gpu:

```
(mldds03) conda uninstall keras
(mldds03) conda install keras-gpu
``` 

Steps:

1. Rename the midi_songs folder to midi_songs_original

2. Rename `weights.hdf5` to `weights.hdf5.original`

3. Create an empty midi_songs folder.  Download about 5-10 .mid files of your choice into it.  For best results, pick a single instrument (e.g. piano).

   Some sources:
   - https://www.8notes.com/scores/1536.asp?ftype=midi
   - http://www.piano-midi.de/midicoll.htm


4. Create a copy of lstm.py called lstm_exercise.py. Edit this file:

   a. Edit the `prepare_sequences` functions to reduce the sequence length from 100 to 25. This will speed up training because  LSTMs can take a long time to learn.  For more details, see: https://github.com/Skuldur/Classical-Piano-Composer/issues/3

   b. Add a TensorBoard callback to monitor training progress.

    ```
    from keras.callbacks import TensorBoard
    from time import time

    ...

    tensorboard = TensorBoard(log_dir='./logs/{}'.format(time()),
                              histogram_freq=0,
                              batch_size=64,
                              write_graph=True)

    ...

    callbacks_list = [checkpoint, tensorboard]

    ```

5. Launch tensorboard:
    ```
   (mldds03) D:\mldds-courseware\03_TextImage\Classical-Piano-Composer>tensorboard --logdir logs --host=0.0.0.0
    
    2018-08-02 15:32:56.198899: I tensorflow/core/platform/cpu_feature_guard.cc:140] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX AVX2
    TensorBoard 1.8.0 at http://0.0.0.0:6006 (Press CTRL+C to quit)
    
    ```
   Open a browser window to http://localhost:6006

6. Start training:
    ```
    python lstm_exercise.py
    ```

If all goes to plan you should see output like this (depending on whether you are running with or without GPU):
   
### CPU-only
```

    (mldds03) D:\mldds-courseware\03_TextImage\Classical-Piano-Composer>python lstm_exercise.py
    Using TensorFlow backend.
    Parsing midi_songs\aidehuhuan.mid
    Parsing midi_songs\kewang.mid
    Parsing midi_songs\parapara.mid
    Parsing midi_songs\shuinenggaoshuwo.mid
    Parsing midi_songs\zhaomi.mid
    2018-08-02 15:35:15.560156: I tensorflow/core/platform/cpu_feature_guard.cc:140] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX AVX2
    Epoch 1/10
    5344/5344 [==============================] - 223s 42ms/step - loss: 4.4112
    Epoch 2/10
     832/5344 [===>..........................] - ETA: 3:06 - loss: 4.1681

```

### GPU

```
    (mldds03) D:\mldds-courseware\03_TextImage\Classical-Piano-Composer>python lstm_exercise.py
    Using TensorFlow backend.
    Parsing midi_songs\aidehuhuan.mid
    Parsing midi_songs\kewang.mid
    Parsing midi_songs\parapara.mid
    Parsing midi_songs\shuinenggaoshuwo.mid
    Parsing midi_songs\zhaomi.mid
    2018-08-02 15:48:55.160750: I C:\users\nwani\_bazel_nwani\mmtm6wb6\execroot\org_tensorflow\tensorflow\core\platform\cpu_feature_guard.cc:140] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX AVX2
    2018-08-02 15:48:55.635377: I C:\users\nwani\_bazel_nwani\mmtm6wb6\execroot\org_tensorflow\tensorflow\core\common_runtime\gpu\gpu_device.cc:1356] Found device 0 with properties:
    name: GeForce GTX 1080 Ti major: 6 minor: 1 memoryClockRate(GHz): 1.582
    pciBusID: 0000:01:00.0
    totalMemory: 11.00GiB freeMemory: 9.01GiB
    2018-08-02 15:48:55.722389: I C:\users\nwani\_bazel_nwani\mmtm6wb6\execroot\org_tensorflow\tensorflow\core\common_runtime\gpu\gpu_device.cc:1356] Found device 1 with properties:
    name: GeForce GTX 1080 Ti major: 6 minor: 1 memoryClockRate(GHz): 1.582
    pciBusID: 0000:02:00.0
    totalMemory: 11.00GiB freeMemory: 9.01GiB
    2018-08-02 15:48:55.727757: I C:\users\nwani\_bazel_nwani\mmtm6wb6\execroot\org_tensorflow\tensorflow\core\common_runtime\gpu\gpu_device.cc:1435] Adding visible gpu devices: 0, 1
    2018-08-02 15:48:57.789074: I C:\users\nwani\_bazel_nwani\mmtm6wb6\execroot\org_tensorflow\tensorflow\core\common_runtime\gpu\gpu_device.cc:923] Device interconnect StreamExecutor with strength 1 edge matrix:
    2018-08-02 15:48:57.793350: I C:\users\nwani\_bazel_nwani\mmtm6wb6\execroot\org_tensorflow\tensorflow\core\common_runtime\gpu\gpu_device.cc:929]      0 1
    2018-08-02 15:48:57.796512: I C:\users\nwani\_bazel_nwani\mmtm6wb6\execroot\org_tensorflow\tensorflow\core\common_runtime\gpu\gpu_device.cc:942] 0:   N N
    2018-08-02 15:48:57.800165: I C:\users\nwani\_bazel_nwani\mmtm6wb6\execroot\org_tensorflow\tensorflow\core\common_runtime\gpu\gpu_device.cc:942] 1:   N N
    2018-08-02 15:48:57.805051: I C:\users\nwani\_bazel_nwani\mmtm6wb6\execroot\org_tensorflow\tensorflow\core\common_runtime\gpu\gpu_device.cc:1053] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 8713 MB memory) -> physical GPU (device: 0, name: GeForce GTX 1080 Ti, pci bus id: 0000:01:00.0, compute capability: 6.1)
    2018-08-02 15:48:58.227726: I C:\users\nwani\_bazel_nwani\mmtm6wb6\execroot\org_tensorflow\tensorflow\core\common_runtime\gpu\gpu_device.cc:1053] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:1 with 8713 MB memory) -> physical GPU (device: 1, name: GeForce GTX 1080 Ti, pci bus id: 0000:02:00.0, compute capability: 6.1)

    Epoch 1/10
    5344/5344 [==============================] - 37s 7ms/step - loss: 4.3308
    Epoch 2/10
    1856/5344 [=========>....................] - ETA: 22s - loss: 4.1595
```

Tensorboard should show the network graph, but it will take some time before a loss curve is shown.

LSTM graph:

![tensorboard](assets/lstm/tensorboard_1.png)

Initial loss values:

![tensorboard](assets/lstm/tensorboard_2.png)

### Predict (again)

Copy the lowest cost weights to weights.hdf5, then run predict.py.

1. copy weights-improvement-<iteration>-<cost>-bigger.hdf5 weights.hdf5
2. python predict.py

In [6]:
play_midi('Classical-Piano-Composer/test_output.mid')

## Appendix

How input and output sequences are generated for LSTM training.

In [2]:
# Annotated versions of the same converter functions in lstm.py

def get_notes(filename):
    """ Get all the notes and chords from a given filename """
    from music21 import converter, instrument, note, chord
    
    notes = []

    midi = converter.parse(filename)

    print("Parsing %s" % file)

    notes_to_parse = None

    try: # file has instrument parts
        s2 = instrument.partitionByInstrument(midi)
        notes_to_parse = s2.parts[0].recurse() 
    except: # file has notes in a flat structure
        notes_to_parse = midi.flat.notes

    for element in notes_to_parse:
        if isinstance(element, note.Note):
            notes.append(str(element.pitch))
        elif isinstance(element, chord.Chord):
            notes.append('.'.join(str(n) for n in element.normalOrder))
    return notes


file = 'Classical-Piano-Composer/midi_songs/appass_1.mid' # change to your midi file
notes = get_notes(file)
print('len(notes):', len(notes))
print(notes)

Parsing Classical-Piano-Composer/midi_songs/appass_1.mid
len(notes): 4876
['C4', 'C2', 'G#3', 'G#1', 'F3', 'F1', 'G#3', 'G#1', 'C4', 'C2', 'F4', 'F2', 'G#4', 'G#2', 'C5', 'C3', 'F5', 'F3', 'F4', 'G#3', 'C5', '0.4.7', 'D5', 'C5', 'D5', '5.8.11', 'E5', 'D5', 'E5', 'D5', 'E5', 'D5', 'E5', 'D5', 'E5', 'D5', 'E5', 'D5', 'C5', 'E5', 'D5', 'C5', '0.4.7', 'C#4', 'C#2', 'B-3', 'B-1', 'F#3', 'F#1', 'B-3', 'B-1', 'C#4', 'C#2', 'F#4', 'F#2', 'B-4', 'B-2', 'C#5', 'C#3', 'F#5', 'F#3', 'F#4', 'B-3', 'C#5', '1.5.8', 'E-5', 'C#5', 'E-5', '6.9.0', 'F5', 'E-5', 'F5', 'E-5', 'F5', 'E-5', 'F5', 'E-5', 'F5', 'E-5', 'F5', 'E-5', 'C#5', 'F5', 'E-5', 'C#5', '1.5.8', 'C5', '4.7', 'D5', 'C5', 'D5', '5.8.11', 'E5', 'D5', 'E5', 'D5', 'E5', 'D5', 'E5', 'D5', 'E5', 'D5', 'E5', 'D5', 'C5', 'E5', 'D5', 'C5', '4.7', 'C#2', 'C#2', 'C#2', 'C2', 'E5', '7.10.1', 'F5', 'F5', '8.10.1', 'G5', 'F5', 'G5', 'F5', 'G5', 'F5', 'G5', 'F5', 'G5', 'F5', 'G5', 'F5', 'E5', 'G5', 'F5', 'E5', '7.10.1', 'C#2', 'C#2', 'C#2', 'C2', '4.7', '

In [3]:
# Annotated versions of the same converter functions in lstm.py

def prepare_sequences(notes, n_vocab):
    """ Prepare the sequences used by the Neural Network """
    sequence_length = 25

    import numpy as np
    from keras.utils import np_utils

    # get all pitch names
    pitchnames = sorted(set(item for item in notes))

     # create a dictionary to map pitches to integers
    note_to_int = dict((note, number) for number, note in enumerate(pitchnames))
    print('note_to_int mapping:', note_to_int)

    network_input = []
    network_output = []

    # create input sequences and the corresponding outputs
    for i in range(0, len(notes) - sequence_length, 1):
        sequence_in = notes[i:i + sequence_length]
        sequence_out = notes[i + sequence_length]
        print('sequence_in', sequence_in)
        print('sequence_out', sequence_out)
        
        network_input.append([note_to_int[char] for char in sequence_in])
        network_output.append(note_to_int[sequence_out])

    n_patterns = len(network_input)
    print('n_patterns:', n_patterns)

    # reshape the input into a format compatible with LSTM layers
    network_input = np.reshape(network_input, (n_patterns, sequence_length, 1))
    print('network_input.shape:', network_input.shape)

    # normalize input
    print('network_input:', network_input)
    network_input = network_input / float(n_vocab)
    print('network_input (normalized to 0-1):', network_input)

    print('output (labels):', network_output)
    network_output = np_utils.to_categorical(network_output)
    print('output (categorical):', network_output)

    return (network_input, network_output)

n_vocab = len(set(notes))
print('n_vocab', n_vocab)
X, y = prepare_sequences(notes, n_vocab)

n_vocab 144


Using TensorFlow backend.


note_to_int mapping: {'0': 0, '0.1': 1, '0.3': 2, '0.3.5': 3, '0.3.6': 4, '0.4': 5, '0.4.7': 6, '0.5': 7, '0.6': 8, '1': 9, '1.3.6': 10, '1.3.7': 11, '1.4': 12, '1.4.7': 13, '1.4.7.10': 14, '1.5': 15, '1.5.7': 16, '1.5.8': 17, '10': 18, '10.0.4': 19, '10.1': 20, '10.1.3': 21, '10.1.5': 22, '11.2': 23, '11.2.5': 24, '11.2.6': 25, '11.3': 26, '11.3.6': 27, '2': 28, '2.5': 29, '2.5.8': 30, '3': 31, '3.5.9': 32, '3.6': 33, '3.6.10': 34, '3.6.8': 35, '3.6.9': 36, '3.7': 37, '3.7.10': 38, '4': 39, '4.7': 40, '4.7.10': 41, '4.7.10.0': 42, '4.8.11': 43, '5': 44, '5.11': 45, '5.7.10': 46, '5.8': 47, '5.8.0': 48, '5.8.11': 49, '5.9.0': 50, '6': 51, '6.10.1': 52, '6.8.0': 53, '6.9': 54, '6.9.0': 55, '7': 56, '7.0': 57, '7.10': 58, '7.10.0': 59, '7.10.1': 60, '7.11.2': 61, '8': 62, '8.0': 63, '8.0.3': 64, '8.1': 65, '8.10.1': 66, '8.11': 67, '8.11.1': 68, '8.11.2': 69, '8.11.3': 70, '9': 71, '9.0': 72, '9.0.3': 73, '9.1': 74, '9.1.4': 75, 'A1': 76, 'A2': 77, 'A3': 78, 'A4': 79, 'A5': 80, 'A6': 81,

sequence_in ['E-6', 'E6', 'E-6', 'E6', 'E-6', 'E6', 'E-6', 'E6', 'E-6', 'E6', 'E-6', 'E6', 'E-6', 'E6', 'E-6', 'E6', 'E-6', 'D6', 'E-6', 'E6', 'E-6', 'C#6', 'B5', 'B-5', 'G#5']
sequence_out G5
sequence_in ['E6', 'E-6', 'E6', 'E-6', 'E6', 'E-6', 'E6', 'E-6', 'E6', 'E-6', 'E6', 'E-6', 'E6', 'E-6', 'E6', 'E-6', 'D6', 'E-6', 'E6', 'E-6', 'C#6', 'B5', 'B-5', 'G#5', 'G5']
sequence_out F#5
sequence_in ['E-6', 'E6', 'E-6', 'E6', 'E-6', 'E6', 'E-6', 'E6', 'E-6', 'E6', 'E-6', 'E6', 'E-6', 'E6', 'E-6', 'D6', 'E-6', 'E6', 'E-6', 'C#6', 'B5', 'B-5', 'G#5', 'G5', 'F#5']
sequence_out E5
sequence_in ['E6', 'E-6', 'E6', 'E-6', 'E6', 'E-6', 'E6', 'E-6', 'E6', 'E-6', 'E6', 'E-6', 'E6', 'E-6', 'D6', 'E-6', 'E6', 'E-6', 'C#6', 'B5', 'B-5', 'G#5', 'G5', 'F#5', 'E5']
sequence_out E-5
sequence_in ['E-6', 'E6', 'E-6', 'E6', 'E-6', 'E6', 'E-6', 'E6', 'E-6', 'E6', 'E-6', 'E6', 'E-6', 'D6', 'E-6', 'E6', 'E-6', 'C#6', 'B5', 'B-5', 'G#5', 'G5', 'F#5', 'E5', 'E-5']
sequence_out C#5
sequence_in ['E6', 'E-6', 'E6', 'E

sequence_in ['B1', 'E-6', 'G#6', 'E-6', 'G#6', 'E-6', 'G#6', 'G#1', 'B3', 'B2', 'G#3', 'G#2', 'E-3', 'E-2', 'B3', 'B2', 'G#3', 'G#2', 'E3', 'E2', 'G#3', 'G#2', 'B3', 'B2', 'E4']
sequence_out E3
sequence_in ['E-6', 'G#6', 'E-6', 'G#6', 'E-6', 'G#6', 'G#1', 'B3', 'B2', 'G#3', 'G#2', 'E-3', 'E-2', 'B3', 'B2', 'G#3', 'G#2', 'E3', 'E2', 'G#3', 'G#2', 'B3', 'B2', 'E4', 'E3']
sequence_out G#4
sequence_in ['G#6', 'E-6', 'G#6', 'E-6', 'G#6', 'G#1', 'B3', 'B2', 'G#3', 'G#2', 'E-3', 'E-2', 'B3', 'B2', 'G#3', 'G#2', 'E3', 'E2', 'G#3', 'G#2', 'B3', 'B2', 'E4', 'E3', 'G#4']
sequence_out G#3
sequence_in ['E-6', 'G#6', 'E-6', 'G#6', 'G#1', 'B3', 'B2', 'G#3', 'G#2', 'E-3', 'E-2', 'B3', 'B2', 'G#3', 'G#2', 'E3', 'E2', 'G#3', 'G#2', 'B3', 'B2', 'E4', 'E3', 'G#4', 'G#3']
sequence_out B4
sequence_in ['G#6', 'E-6', 'G#6', 'G#1', 'B3', 'B2', 'G#3', 'G#2', 'E-3', 'E-2', 'B3', 'B2', 'G#3', 'G#2', 'E3', 'E2', 'G#3', 'G#2', 'B3', 'B2', 'E4', 'E3', 'G#4', 'G#3', 'B4']
sequence_out B3
sequence_in ['E-6', 'G#6', 'G

...

sequence_in ['G2', 'B-2', 'B-2', 'C#3', 'E3', 'E2', 'G2', 'G2', 'B-2', 'C#3', '7.10', 'C#2', 'E3', '7.10', 'E3', '7.10', 'E3', '7.10', 'C#5', 'E3', '7.10', 'C#5', 'E3', '7.10', 'C#5']
sequence_out E3
sequence_in ['B-2', 'B-2', 'C#3', 'E3', 'E2', 'G2', 'G2', 'B-2', 'C#3', '7.10', 'C#2', 'E3', '7.10', 'E3', '7.10', 'E3', '7.10', 'C#5', 'E3', '7.10', 'C#5', 'E3', '7.10', 'C#5', 'E3']
sequence_out 7.10
sequence_in ['B-2', 'C#3', 'E3', 'E2', 'G2', 'G2', 'B-2', 'C#3', '7.10', 'C#2', 'E3', '7.10', 'E3', '7.10', 'E3', '7.10', 'C#5', 'E3', '7.10', 'C#5', 'E3', '7.10', 'C#5', 'E3', '7.10']
sequence_out C#5
sequence_in ['C#3', 'E3', 'E2', 'G2', 'G2', 'B-2', 'C#3', '7.10', 'C#2', 'E3', '7.10', 'E3', '7.10', 'E3', '7.10', 'C#5', 'E3', '7.10', 'C#5', 'E3', '7.10', 'C#5', 'E3', '7.10', 'C#5']
sequence_out E3
sequence_in ['E3', 'E2', 'G2', 'G2', 'B-2', 'C#3', '7.10', 'C#2', 'E3', '7.10', 'E3', '7.10', 'E3', '7.10', 'C#5', 'E3', '7.10', 'C#5', 'E3', '7.10', 'C#5', 'E3', '7.10', 'C#5', 'E3']
sequence_ou

sequence_out G#5
sequence_in ['1.4.7.10', '4.7', '1.4.7.10', '7.10', '1.4.7.10', '10.1', '1.4.7.10', '1.4', '1.4.7.10', '4.7', 'G5', 'E5', '10.1', 'G#5', 'F5', 'G#5', '11.2', 'B-5', 'G#5', 'B-5', 'G#5', 'B-5', 'G#5', 'B-5', 'G#5']
sequence_out G5
sequence_in ['4.7', '1.4.7.10', '7.10', '1.4.7.10', '10.1', '1.4.7.10', '1.4', '1.4.7.10', '4.7', 'G5', 'E5', '10.1', 'G#5', 'F5', 'G#5', '11.2', 'B-5', 'G#5', 'B-5', 'G#5', 'B-5', 'G#5', 'B-5', 'G#5', 'G5']
sequence_out B-5
sequence_in ['1.4.7.10', '7.10', '1.4.7.10', '10.1', '1.4.7.10', '1.4', '1.4.7.10', '4.7', 'G5', 'E5', '10.1', 'G#5', 'F5', 'G#5', '11.2', 'B-5', 'G#5', 'B-5', 'G#5', 'B-5', 'G#5', 'B-5', 'G#5', 'G5', 'B-5']
sequence_out G#5
sequence_in ['7.10', '1.4.7.10', '10.1', '1.4.7.10', '1.4', '1.4.7.10', '4.7', 'G5', 'E5', '10.1', 'G#5', 'F5', 'G#5', '11.2', 'B-5', 'G#5', 'B-5', 'G#5', 'B-5', 'G#5', 'B-5', 'G#5', 'G5', 'B-5', 'G#5']
sequence_out 4.7
sequence_in ['1.4.7.10', '10.1', '1.4.7.10', '1.4', '1.4.7.10', '4.7', 'G5', 'E5', 

sequence_out C#3
sequence_in ['C#3', 'F4', 'F3', 'G#4', 'C#3', 'C#5', 'F2', 'G#4', 'C#3', 'F5', 'F2', 'G#4', 'C#3', 'E-5', 'F2', 'G#4', 'C#3', 'C#5', 'F2', 'G#4', 'C#3', 'C#5', 'F2', 'G#4', 'C#3']
sequence_out F4
sequence_in ['F4', 'F3', 'G#4', 'C#3', 'C#5', 'F2', 'G#4', 'C#3', 'F5', 'F2', 'G#4', 'C#3', 'E-5', 'F2', 'G#4', 'C#3', 'C#5', 'F2', 'G#4', 'C#3', 'C#5', 'F2', 'G#4', 'C#3', 'F4']
sequence_out F3
sequence_in ['F3', 'G#4', 'C#3', 'C#5', 'F2', 'G#4', 'C#3', 'F5', 'F2', 'G#4', 'C#3', 'E-5', 'F2', 'G#4', 'C#3', 'C#5', 'F2', 'G#4', 'C#3', 'C#5', 'F2', 'G#4', 'C#3', 'F4', 'F3']
sequence_out G#4
sequence_in ['G#4', 'C#3', 'C#5', 'F2', 'G#4', 'C#3', 'F5', 'F2', 'G#4', 'C#3', 'E-5', 'F2', 'G#4', 'C#3', 'C#5', 'F2', 'G#4', 'C#3', 'C#5', 'F2', 'G#4', 'C#3', 'F4', 'F3', 'G#4']
sequence_out C#3
sequence_in ['C#3', 'C#5', 'F2', 'G#4', 'C#3', 'F5', 'F2', 'G#4', 'C#3', 'E-5', 'F2', 'G#4', 'C#3', 'C#5', 'F2', 'G#4', 'C#3', 'C#5', 'F2', 'G#4', 'C#3', 'F4', 'F3', 'G#4', 'C#3']
sequence_out C#5
se

sequence_out G4
sequence_in ['E-5', 'A5', 'C6', 'A6', 'E-6', 'C7', '1', 'F4', 'C#4', 'B-4', 'F3', 'B-3', 'C#4', 'B-4', 'F4', 'C#5', '4', 'G4', 'E4', 'C5', 'G3', 'C4', 'E4', 'C5', 'G4']
sequence_out E5
sequence_in ['A5', 'C6', 'A6', 'E-6', 'C7', '1', 'F4', 'C#4', 'B-4', 'F3', 'B-3', 'C#4', 'B-4', 'F4', 'C#5', '4', 'G4', 'E4', 'C5', 'G3', 'C4', 'E4', 'C5', 'G4', 'E5']
sequence_out 5
sequence_in ['C6', 'A6', 'E-6', 'C7', '1', 'F4', 'C#4', 'B-4', 'F3', 'B-3', 'C#4', 'B-4', 'F4', 'C#5', '4', 'G4', 'E4', 'C5', 'G3', 'C4', 'E4', 'C5', 'G4', 'E5', '5']
sequence_out G#4
sequence_in ['A6', 'E-6', 'C7', '1', 'F4', 'C#4', 'B-4', 'F3', 'B-3', 'C#4', 'B-4', 'F4', 'C#5', '4', 'G4', 'E4', 'C5', 'G3', 'C4', 'E4', 'C5', 'G4', 'E5', '5', 'G#4']
sequence_out F4
sequence_in ['E-6', 'C7', '1', 'F4', 'C#4', 'B-4', 'F3', 'B-3', 'C#4', 'B-4', 'F4', 'C#5', '4', 'G4', 'E4', 'C5', 'G3', 'C4', 'E4', 'C5', 'G4', 'E5', '5', 'G#4', 'F4']
sequence_out C#5
sequence_in ['C7', '1', 'F4', 'C#4', 'B-4', 'F3', 'B-3', 'C#4',

sequence_out C4
sequence_in ['G#3', 'C4', 'G#3', 'C5', 'C4', 'F5', 'G#3', 'C4', 'G#3', 'C4', 'G#3', 'C4', 'G#5', 'G#3', 'C4', 'G#3', 'C4', 'G#3', 'C6', 'C4', 'F6', 'G#3', 'C4', 'G#3', 'C4']
sequence_out G#3
sequence_in ['C4', 'G#3', 'C5', 'C4', 'F5', 'G#3', 'C4', 'G#3', 'C4', 'G#3', 'C4', 'G#5', 'G#3', 'C4', 'G#3', 'C4', 'G#3', 'C6', 'C4', 'F6', 'G#3', 'C4', 'G#3', 'C4', 'G#3']
sequence_out C4
sequence_in ['G#3', 'C5', 'C4', 'F5', 'G#3', 'C4', 'G#3', 'C4', 'G#3', 'C4', 'G#5', 'G#3', 'C4', 'G#3', 'C4', 'G#3', 'C6', 'C4', 'F6', 'G#3', 'C4', 'G#3', 'C4', 'G#3', 'C4']
sequence_out C6
sequence_in ['C5', 'C4', 'F5', 'G#3', 'C4', 'G#3', 'C4', 'G#3', 'C4', 'G#5', 'G#3', 'C4', 'G#3', 'C4', 'G#3', 'C6', 'C4', 'F6', 'G#3', 'C4', 'G#3', 'C4', 'G#3', 'C4', 'C6']
sequence_out G#3
sequence_in ['C4', 'F5', 'G#3', 'C4', 'G#3', 'C4', 'G#3', 'C4', 'G#5', 'G#3', 'C4', 'G#3', 'C4', 'G#3', 'C6', 'C4', 'F6', 'G#3', 'C4', 'G#3', 'C4', 'G#3', 'C4', 'C6', 'G#3']
sequence_out C4
sequence_in ['F5', 'G#3', 'C4', '